In [1178]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil,subprocess
import os
import subprocess
import re
import mzbatch
from pyopenms import MSExperiment,MzMLFile,PeakPickerHiRes
import time
import pyopenms
from pyopenms import Plotting
import importlib as imp
imp.reload(mzbatch)

<module 'mzbatch' from '/mnt/nas231/资料备份/LQ/JupyterPro/Fulaoshi/mzbatch.py'>

Statistics on all MS/MS spectra

In [1132]:
def get_246filename_list(filepre_list):
    filename_list = []
    ev_list = ['1', '10', '100', '15', '20', '25', '30', '35', '40', '50', '60','70', '80', '204060']
    for filepre in filepre_list:
        for ev in ev_list:
            for tail in ['pos','neg']:
                filename_list.append(f'{filepre}_NCE{ev}_{tail}')
    return filename_list
filename_list = get_246filename_list(['1867-73-8'])
str(filename_list)

"['1867-73-8_NCE1_pos', '1867-73-8_NCE1_neg', '1867-73-8_NCE10_pos', '1867-73-8_NCE10_neg', '1867-73-8_NCE100_pos', '1867-73-8_NCE100_neg', '1867-73-8_NCE15_pos', '1867-73-8_NCE15_neg', '1867-73-8_NCE20_pos', '1867-73-8_NCE20_neg', '1867-73-8_NCE25_pos', '1867-73-8_NCE25_neg', '1867-73-8_NCE30_pos', '1867-73-8_NCE30_neg', '1867-73-8_NCE35_pos', '1867-73-8_NCE35_neg', '1867-73-8_NCE40_pos', '1867-73-8_NCE40_neg', '1867-73-8_NCE50_pos', '1867-73-8_NCE50_neg', '1867-73-8_NCE60_pos', '1867-73-8_NCE60_neg', '1867-73-8_NCE70_pos', '1867-73-8_NCE70_neg', '1867-73-8_NCE80_pos', '1867-73-8_NCE80_neg', '1867-73-8_NCE204060_pos', '1867-73-8_NCE204060_neg']"

In [ ]:
new_columns = ['batch', 'sampleid', 'CAS No.', 'EV', 'pos_neg', 'mz', 'Product Name', 'filename'
               , 'refer_valid', 'ref_rt', 'ref_intensity','is_max_ms1'
               , 'ms1_intensity','is_max_mol', 'mol_intensity','is_max_base', 'base_intensity']

pre_batch = ""
for batch_sampleid in batch_sampleid_list[320:321]:
    MS2_info_df=pd.DataFrame(columns=['batch','sampleid','CAS No.','EV','pos_neg','mz','Product Name','filename'
                                      , 'refer_valid', 'ref_rt', 'ref_intensity'
                                      , 'ms1_intensity', 'mol_intensity', 'base_intensity'])
    n=0
    
    batch,sampleid = batch_sampleid.split('/')
    
    if pre_batch != batch:
        print(f'==={batch}===')
        pre_batch=batch
    temp_ref_df = ref_all_df[ref_all_df['batch_sampleid'] == batch_sampleid]
    cas_list = temp_ref_df['CAS No.'].tolist()
    foldername_list = get_246filename_list(cas_list) #['1867-73-8_NCE1_pos']
    
    for foldername in foldername_list:
        
        foldername_split_gang_list = foldername.split('_')
        cas, ev, pos_neg = foldername_split_gang_list[0],foldername_split_gang_list[1][3:],foldername_split_gang_list[2]
        
        mask = ((temp_ref_df['batch']==batch) & (temp_ref_df['sampleid']==sampleid) & (temp_ref_df['CAS No.']==cas) & (temp_ref_df['pos_neg']==pos_neg))
        if temp_ref_df[mask].shape[0]==0:
            continue
        pro_name, refer_valid,theo_mz,ref_rt,ref_intensity = temp_ref_df[mask].reset_index(drop=True).loc[0,
                                                                           ['Product Name'
                                                                           ,'refer_RT', 'm/z'
                                                                           ,'RT time', 'NL']]
        if pd.isna(ref_rt):
            continue
        
        folderpath = f'{out_root_path}/{batch}/NCE_compound/{sampleid}/{foldername}'
        if not os.path.exists(folderpath):
            print('folderpath does not exist',folderpath)
            continue
        
        ms2_filename_list = mzbatch.get_ms2_file(folderpath)
        if ms2_filename_list == -1:
            ms2_filename = False
            MS2_info_df.loc[n]=[batch, sampleid, cas, ev, pos_neg, theo_mz, pro_name,ms2_filename
                                , refer_valid, ref_rt, ref_intensity
                                , -1, -1, -1]
            n+=1
            continue
        for ms2_filename in ms2_filename_list:            
            ms2_filename_gang_list = ms2_filename.split('_')
            ms2_rt = float(ms2_filename_gang_list[-1][:-4])
            ms1_file = f'{folderpath}/{foldername}_MS1.csv'
            ms1_spec = pd.read_csv(ms1_file,index_col=0)
            ms2_filename_gang_list = ms2_filename.split('_')
            ms2_rt = float(ms2_filename_gang_list[-1][:-4])
            ms1_intensity = ms1_spec[ms1_spec['rt'].values <= ms2_rt].iloc[-1,1]
            
            ms2_file = f'{folderpath}/{ms2_filename}'
            intensity_df = pd.read_csv(ms2_file,index_col=0)
            
            mz_uplimit = mzbatch.get_mz_uplimit(theo_mz)
            intensity_df = intensity_df[intensity_df['Mass']<=mz_uplimit]

            basepeak_mass,basepeak_intensity = mzbatch.get_basepeak(intensity_df,theo_mz)

            molpeak_mass,molpeak_intensity = mzbatch.get_real_val(intensity_df,theo_mz)

            MS2_info_df.loc[n]=[batch, sampleid, cas, ev, pos_neg, theo_mz, pro_name,ms2_filename
                                ,refer_valid,ref_rt,ref_intensity
                                ,ms1_intensity,molpeak_intensity,basepeak_intensity]
            n+=1
            # break
    print(MS2_info_df.shape)
    max_ms1_indexes = MS2_info_df.groupby(by=['CAS No.','EV','pos_neg'])['ms1_intensity'].idxmax()
    MS2_info_df['is_max_ms1']=0
    MS2_info_df.loc[max_ms1_indexes,'is_max_ms1']=1
    MS2_info_df.loc[MS2_info_df[MS2_info_df["ms1_intensity"]==-1].index.to_list(),"is_max_ms1"]=0
    MS2_info_df.loc[MS2_info_df[MS2_info_df["ms1_intensity"]==0].index.to_list(),"is_max_ms1"]=0

    max_mol_indexes = MS2_info_df.grou


===700_Amide===
(13670, 14)


In [663]:
max_ms1_indexes = MS2_info_df.groupby(by=['CAS No.','EV','pos_neg'])['ms1_intensity'].idxmax()
MS2_info_df['is_max_ms1']=0
MS2_info_df.loc[max_ms1_indexes,'is_max_ms1']=1
MS2_info_df.loc[MS2_info_df[MS2_info_df["ms1_intensity"]==-1].index.to_list(),"is_max_ms1"]=0
MS2_info_df.loc[MS2_info_df[MS2_info_df["ms1_intensity"]==0].index.to_list(),"is_max_ms1"]=0

# Check if the file does not exist and the result is empty

In [ ]:
for batch_sampleid in batch_sampleid_list:
    batch,sampleid = batch_sampleid.split('/')
    
    info_file = f'{out_root_path}/{batch}/NCE_MS2_info/MS2_info_df_{sampleid}.csv'
    if not os.path.exists(info_file):
        print(f'does not exist: {batch} == {batch_sampleid_list.index(batch_sampleid)} == info_df_{sampleid}.csv')
        continue
        
    one_info_df = pd.read_csv(f'{info_file}',index_col=0)
    if one_info_df.shape[0]==0:
        print(f'empty: {batch} == info_df_{sampleid}.csv')


# The info results of each batch are merged and eventually merged into the root directory info_all

In [1417]:
nums_sum = 0
for batch in batch_list:
    MS2_info_path = f'{out_root_path}/{batch}/NCE_MS2_info'
    outfile = f'{out_root_path}/{batch}/MS2_info_all_df.csv'
    
    info_df_list = os.listdir(MS2_info_path)

    merged_data_df = pd.DataFrame()
    for info_df_file in info_df_list:
        info_df_file_path =  f'{MS2_info_path}/{info_df_file}'
        one_info_df = pd.read_csv(info_df_file_path,index_col=0)
        merged_data_df = pd.concat([merged_data_df, one_info_df], ignore_index=True)
    nums_sum += merged_data_df.shape[0]
    # merged_data_df.to_excel(outfile)
    merged_data_df.to_csv(outfile, encoding='utf-8-sig')   
    print(f'batch-({batch}) shape: {merged_data_df.shape}, sum: {nums_sum}  ')

batch-(700_Amide) shape: (126468, 19), sum: 126468  
batch-(700_C18) shape: (138760, 19), sum: 265228  
batch-(873_Amide) shape: (157431, 19), sum: 422659  
batch-(873_C18) shape: (168145, 19), sum: 590804  
batch-(3545_Amide) shape: (910668, 19), sum: 1501472  
batch-(3545_C18) shape: (1097830, 19), sum: 2599302  
batch-(BC_Amide) shape: (11663, 19), sum: 2610965  
batch-(BC_C18) shape: (4928, 19), sum: 2615893  


In [1418]:
concat_info_all_df = pd.DataFrame(columns=merged_data_df.columns)
for batch in batch_list:
    # batch_info_all_df = pd.read_excel(f'{out_root_path}/{batch}/MS2_info_all_df.xlsx',index_col=0)
    batch_info_all_df = pd.read_csv(f'{out_root_path}/{batch}/MS2_info_all_df.csv',index_col=0)
    concat_info_all_df = pd.concat([concat_info_all_df,batch_info_all_df])
    # break
concat_info_all_df.to_csv(f'{out_root_path}/concat_MS2_info_all_df.csv',encoding='utf-8-sig')
concat_info_all_df.to_pickle(f'{out_root_path}/concat_MS2_info_all_df.pkl')

/tmp/ipykernel_2411273/2882656136.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  batch_info_all_df = pd.read_csv(f'{out_root_path}/{batch}/MS2_info_all_df.csv',index_col=0)
/tmp/ipykernel_2411273/2882656136.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  batch_info_all_df = pd.read_csv(f'{out_root_path}/{batch}/MS2_info_all_df.csv',index_col=0)
/tmp/ipykernel_2411273/2882656136.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  batch_info_all_df = pd.read_csv(f'{out_root_path}/{batch}/MS2_info_all_df.csv',index_col=0)
/tmp/ipykernel_2411273/2882656136.py:4: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  batch_info_all_df = pd.read_csv(f'{out_root_path}/{batch}/MS2_info_all_df.csv',index_col=0)


In [1419]:
concat_info_all_df.shape

(2615893, 19)

# Filter 39 spectra for each compound

In [1446]:
# 读取所有ms2
# concat_info_all_df = pd.read_pickle(f'{out_root_path}/concat_MS2_info_all_df.pkl')
concat_info_all_df = pd.read_pickle('../../Fulaoshi/230821_mzML_res/rerun240410/concat_MS2_info_all_df.pkl')
print("原始大小：", concat_info_all_df.shape)
concat_info_all_df=concat_info_all_df.reset_index(drop=True)
# 删除False
concat_info_all_df = concat_info_all_df[concat_info_all_df["filename"]!="False"]
concat_info_all_df['bscpe']=concat_info_all_df['batch']+'+'+concat_info_all_df['sampleid'].astype(str)+'+'+concat_info_all_df['CAS No.']+'+'+concat_info_all_df['pos_neg']+'+'+concat_info_all_df['EV'].astype(str)
# 增加基峰和母峰倍数
concat_info_all_df["ms1_base_rate"] = concat_info_all_df["ms1_intensity"]/concat_info_all_df["base_intensity"]
concat_info_all_df["ms1_mol_rate"] = concat_info_all_df["ms1_intensity"]/concat_info_all_df["mol_intensity"]
print(concat_info_all_df.shape)
concat_info_all_df.head(2)

原始大小： (2615893, 19)
(2614930, 20)


,batch,sampleid,CAS No.,EV,pos_neg,mz,Product Name,filename,refer_valid,ref_rt,ref_intensity,is_max_ms1,ms1_intensity,is_max_mol,mol_intensity,is_max_base,base_intensity,ms1_base_rate,ms1_mol_rate,bscpe
0,700_Amide,S98,576-42-1,1,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE1_neg_MS2_12037_802.39.csv,1,826.2,494000.0,0,6.903690e+05,0,488907.75,0,488907.75,1.412064,1.412064,700_Amide+S98+576-42-1+neg+1
1,700_Amide,S98,576-42-1,1,neg,209.03029,D-Glucaric acid (potassium),576-42-1_NCE1_neg_MS2_12081_805.6.csv,1,826.2,494000.0,0,1.036211e+06,0,752876.25,0,752876.25,1.376337,1.376337,700_Amide+S98+576-42-1+neg+1


In [1593]:
# 过滤39张图
con39_info_df = concat_info_all_df[(concat_info_all_df['is_max_ms1']==1) 
                                   | (concat_info_all_df['is_max_mol']==1) 
                                   | (concat_info_all_df['is_max_base']==1)].copy()

In [1596]:
def get_bscep_filename_list(bscp):
    ev_list = ['1', '10', '100', '15', '20', '25', '30', '35', '40', '50', '60','70', '80', '204060']
    return [f"{bscp}+{ev}" for ev in ev_list]
get_bscep_filename_list(bscp_list[0])

['873_Amide+26386+1867-73-8+pos+1',
 '873_Amide+26386+1867-73-8+pos+10',
 '873_Amide+26386+1867-73-8+pos+100',
 '873_Amide+26386+1867-73-8+pos+15',
 '873_Amide+26386+1867-73-8+pos+20',
 '873_Amide+26386+1867-73-8+pos+25',
 '873_Amide+26386+1867-73-8+pos+30',
 '873_Amide+26386+1867-73-8+pos+35',
 '873_Amide+26386+1867-73-8+pos+40',
 '873_Amide+26386+1867-73-8+pos+50',
 '873_Amide+26386+1867-73-8+pos+60',
 '873_Amide+26386+1867-73-8+pos+70',
 '873_Amide+26386+1867-73-8+pos+80',
 '873_Amide+26386+1867-73-8+pos+204060']

In [1597]:
bscp_list = ref_all_df['bscp'].unique().tolist()
bscpe_all_list = []
[bscpe_all_list.extend(get_bscep_filename_list(bscp)) for bscp in bscp_list]
# bscpe_all_list = sum(bscpe_all_list,[])
len(bscpe_all_list)

191352

In [1601]:
print(con39_info_df[(con39_info_df["ms1_base_rate"]>=100) & (con39_info_df["ms1_mol_rate"]>=500)].shape)
bscpe_100_500_list = con39_info_df[(con39_info_df["ms1_base_rate"]>=100) & (con39_info_df["ms1_mol_rate"]>=500)]["bscpe"].unique().tolist()
len(bscpe_100_500_list)

(320, 20)


303

In [1602]:
len(bscpe_100_500_list)

303

In [1604]:
# bscpe_100_500_list.index("700_Amide+S89+74-81-7+neg+80")
bscpe = "700_Amide+S89+74-81-7+neg+80"
con39_nums = con39_info_df[con39_info_df["bscpe"]==bscpe].shape[0]
con39_nums
con39_100_500_nums = con39_info_df[(con39_info_df["bscpe"]==bscpe) & (con39_info_df["ms1_base_rate"]>=100) & (con39_info_df["ms1_mol_rate"]>=500)].shape[0]
con39_100_500_nums

1

In [ ]:
con39_info_df2 = con39_info_df[(con39_info_df['is_max_ms1']==1) 
                                   | (con39_info_df['is_max_mol']==1) 
                                   | (con39_info_df['is_max_base']==1)].copy()
con39_info_df2.to_excel(f'{out_root_path}/con39_info_df_39_figures_filtered_100500.xlsx')


Exception ignored in: <function ZipFile.__del__ at 0x7f56557284c0>
Traceback (most recent call last):
  File "/home/panlab/Softwares/miniconda3/envs/py38_lq/lib/python3.8/zipfile.py", line 1821, in __del__
    self.close()
  File "/home/panlab/Softwares/miniconda3/envs/py38_lq/lib/python3.8/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [1617]:
con39_info_df2[con39_info_df2['bscpe']==bscpe]
con39_bscpe_dict = dict(con39_info_df2['bscpe'].value_counts())
bscpe_all_dict={}
for bscpe in bscpe_all_list:
    if bscpe in con39_bscpe_dict.keys():        
        bscpe_all_dict[bscpe] = con39_bscpe_dict[bscpe]
    else:
        bscpe_all_dict[bscpe] = 0
len(bscpe_all_dict)

191352

In [1618]:
bscpe_count_df = pd.DataFrame.from_dict(bscpe_all_dict, orient='index', columns=['value'])
bscpe_count_df['key'] = bscpe_count_df.index
bscpe_count_df = bscpe_count_df[['key', 'value']]
bscpe_count_df=bscpe_count_df.reset_index(drop=True)
bscpe_count_df.to_csv(f'{out_root_path}/bscpe_count_df.csv')
bscpe_count_df.head()

,key,value
0,873_Amide+26386+1867-73-8+pos+1,1
1,873_Amide+26386+1867-73-8+pos+10,1
2,873_Amide+26386+1867-73-8+pos+100,2
3,873_Amide+26386+1867-73-8+pos+15,1
4,873_Amide+26386+1867-73-8+pos+20,1


In [1619]:
bscpe_all_zero_keys = [key for key, value in bscpe_all_dict.items() if value == 0]
print(len(bscpe_all_zero_keys),bscpe_all_zero_keys[0:5])

566 ['873_Amide+26386+2078-54-8+pos+204060', '873_Amide+26386+150-13-0+neg+204060', '873_Amide+26386+56-47-3+neg+204060', '873_Amide+26386+1477-68-5+neg+1', '873_Amide+26386+1477-68-5+neg+10']


In [1622]:
max_ms1_39_indexes = con39_info_df2.groupby(by=['bscpe'])['ms1_intensity'].idxmax()
con39_info_df2['is_max_ms1_39']=0
con39_info_df2.loc[max_ms1_39_indexes,'is_max_ms1_39']=1


In [1624]:
con1_info_df = con39_info_df2[con39_info_df2["is_max_ms1_39"]==1]

In [1626]:
con1_info_df.shape

(190786, 21)

In [1628]:
intersection_set = set(bscpe_all_list).intersection(set(con1_info_df["bscpe"].unique().tolist()))
len(intersection_set)

left_difference_list = list(set(bscpe_all_list) - set(con1_info_df["bscpe"].unique().tolist()))
print(len(left_difference_list),left_difference_list[0:5])

right_difference_list = list(set(con1_info_df["bscpe"].unique().tolist()) - set(bscpe_all_list))
print(len(right_difference_list),right_difference_list[0:5])

566 ['873_C18+26395+6038-19-3+neg+15', '873_C18+26388+89-57-6+pos+80', '873_Amide+26386+1477-68-5+neg+20', '873_Amide+26391+26020-14-4+neg+20', '873_Amide+3+87-99-0+pos+50']
0 []


In [1629]:
from collections import Counter
ref_all_df['scp']=ref_all_df['sepuzhu'].astype(str)+'+'+ref_all_df['CAS No.']+'+'+ref_all_df['pos_neg']
scp_list = ref_all_df['scp'].tolist()
scp_dict = dict(Counter(scp_list))
scp_repeat_list = [key for key, value in scp_dict.items() if value >= 2]
print(len(scp_repeat_list))


154


In [1632]:
con1_info_df["filepath"] = con1_info_df['batch']+"/NCE_compound/"+con1_info_df['sampleid'].astype(str)+"/"+con1_info_df['CAS No.']+"_NCE"+con1_info_df['EV'].astype(str)+"_"+con1_info_df['pos_neg']+"/"+con1_info_df['filename']
con1_info_df["filter_filepath"] = con1_info_df['batch']+"/NCE_compound_filter/"+con1_info_df['sampleid'].astype(str)+"/"+con1_info_df['CAS No.']+"_NCE"+con1_info_df['EV'].astype(str)+"_"+con1_info_df['pos_neg']+"/"+con1_info_df['filename']
con1_info_df.to_excel(f'{out_root_path}/con1_info_df.xlsx')

/tmp/ipykernel_2411273/556206305.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con1_info_df["filepath"] = con1_info_df['batch']+"/NCE_compound/"+con1_info_df['sampleid'].astype(str)+"/"+con1_info_df['CAS No.']+"_NCE"+con1_info_df['EV'].astype(str)+"_"+con1_info_df['pos_neg']+"/"+con1_info_df['filename']
/tmp/ipykernel_2411273/556206305.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  con1_info_df["filter_filepath"] = con1_info_df['batch']+"/NCE_compound_filter/"+con1_info_df['sampleid'].astype(str)

In [1634]:
con1_info_df.shape

(190786, 23)

In [362]:
con39_info_df = concat_info_all_df[(concat_info_all_df['is_max_ms1']==1) 
                                   | (concat_info_all_df['is_max_mol']==1) 
                                   | (concat_info_all_df['is_max_base']==1)]
con39_info_df = con39_info_df[con39_info_df["filename"] != "False"]
con39_info_df["filepath"] = con39_info_df['batch']+"/NCE_compound/"+con39_info_df['sampleid'].astype(str)+"/"+con39_info_df['CAS No.']+"_NCE"+con39_info_df['EV'].astype(str)+"_"+con39_info_df['pos_neg']+"/"+con39_info_df['filename']
# con39_info_df.to_excel(f'{out_root_path}/con39_info_df_39张图去除100倍和500倍.xlsx')

# Build the subsequent similarity calculation reference library smi_ref_df.xlsx and add 39 file names to it

In [693]:
ref_all_df = pd.read_excel("../../Fulaoshi/230821_mzML/ref_all.xlsx",sheet_name="concat",index_col=0)
ref_all_df['batch_sampleid'] = ref_all_df['batch']+'/'+ref_all_df['sampleid']
ref_all_df.head()

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,M.Wt,pos_neg,refer_RT,m/z,RT time,NL,bscp,Adduct,batch_sampleid
0,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.11960,196.2,117000000,873_Amide+26386+1867-73-8+pos,[M+H]+,873_Amide/26386
1,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,neg,1,280.10512,206.4,8250000,873_Amide+26386+1867-73-8+neg,[M-H]-,873_Amide/26386
2,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,pos,1,347.22160,63.6,25400000,873_Amide+26386+50-22-6+pos,[M+H]+,873_Amide/26386
3,873_Amide,Amide,26386,Corticosterone,50-22-6,C21H30O4,346.46,neg,1,345.20713,63.6,3360000,873_Amide+26386+50-22-6+neg,[M-H]-,873_Amide/26386
4,873_Amide,Amide,26386,Hexylresorcinol,136-77-6,C12H18O2,194.27,neg,1,193.12340,57.0,27500000,873_Amide+26386+136-77-6+neg,[M-H]-,873_Amide/26386


In [695]:
smi_ref_df = pd.DataFrame(columns=['batch', 'sepuzhu', 'sampleid', 'Product Name', 'CAS No.', 'Formula', 'MW'
                      , 'pos_neg', 'is_valid', 'mz', 'ref_rt', 'NL', 'bscp','batch_sampleid'])
smi_ref_df

,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,MW,pos_neg,is_valid,mz,ref_rt,NL,bscp,batch_sampleid


In [699]:
ref_all_df.columns = ['batch', 'sepuzhu', 'sampleid', 'Product Name', 'CAS No.', 'Formula', 'MW', 'pos_neg', 'is_valid', 'mz', 'ref_rt', 'NL', 'bscp','batch_sampleid',"Adduct"]
smi_ref_df = ref_all_df.iloc[:,0:14]

In [701]:
smi_ref_df.to_excel(f'{root_path}/smi_ref_df.xlsx')

In [141]:
smi_ref_df = pd.read_excel(f"{root_path}/smi_ref_df.xlsx",index_col=0)
smi_ref_df['bscp']=smi_ref_df['batch']+'+'+smi_ref_df['sampleid'].astype(str)+'+'+smi_ref_df['CAS No.']+'+'+smi_ref_df['pos_neg']
bscp_list = smi_ref_df['bscp'].unique().tolist()
print(smi_ref_df.shape)
smi_ref_df.head(2)

(13671, 13)


,batch,sepuzhu,sampleid,Product Name,CAS No.,Formula,MW,pos_neg,is_valid,mz,ref_rt,NL,bscp
0,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,pos,1,282.11960,196.2,117000000,873_Amide+26386+1867-73-8+pos
1,873_Amide,Amide,26386,N6-Methyladenosine,1867-73-8,C11H15N5O4,281.27,neg,1,280.10512,206.4,8250000,873_Amide+26386+1867-73-8+neg


In [148]:
filename_2list = []
for bscp in bscp_list:
    filename_list = con39_info_df[(con39_info_df["bscp"] == bscp) & (con39_info_df["filename"] != "False")]["filename"].to_list()
    filename_2list.append(filename_list)
smi_ref_df["filename_list"] = filename_2list

In [149]:
smi_ref_df.to_csv(f"{root_path}/smi_ref_df_filenamelist.csv")